In [4]:
from transformers import SegformerForSemanticSegmentation
import torch


encoder_2d = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/mit-b1",
    num_labels=1,
    ignore_mismatched_sizes=True,
    num_channels=1
).eval()




dummy_input = torch.randn(1, 1, 256, 256)
y_out = encoder_2d(dummy_input)
y_out.logits.shape

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b1 and are newly initialized: ['decode_head.linear_c.1.proj.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.weight', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.batch_norm.running_mean', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.classifier.bias', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b1 and are newly initialized because the shapes did not m

torch.Size([1, 1, 64, 64])

In [2]:
encoder_2d.config

SegformerConfig {
  "_name_or_path": "nvidia/mit-b1",
  "architectures": [
    "SegformerForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout_prob": 0.1,
  "decoder_hidden_size": 256,
  "depths": [
    2,
    2,
    2,
    2
  ],
  "downsampling_rates": [
    1,
    4,
    8,
    16
  ],
  "drop_path_rate": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_sizes": [
    64,
    128,
    320,
    512
  ],
  "id2label": {
    "0": "LABEL_0"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-06,
  "mlp_ratios": [
    4,
    4,
    4,
    4
  ],
  "model_type": "segformer",
  "num_attention_heads": [
    1,
    2,
    5,
    8
  ],
  "num_channels": 1,
  "num_encoder_blocks": 4,
  "patch_sizes": [
    7,
    3,
    3,
    3
  ],
  "reshape_last_stage": true,
  "semantic_loss_ignore_index": 255,
  "sr_ratios": [
    8,
    4,
    2,
    1
  ],
  "strides": [
    4,
    2,

In [13]:
import torch.nn as nn

class CustomModel(nn.Module):

	def __init__(self, scale_factor=2):
		super().__init__()
		
		self.upsampler = nn.Upsample(scale_factor=scale_factor, mode='bilinear', align_corners=True)
		self.encoder = SegformerForSemanticSegmentation.from_pretrained(
			"nvidia/mit-b1",
			num_labels=1,
			ignore_mismatched_sizes=True,
			num_channels=1
		)
		self.upscaler1 = nn.ConvTranspose2d(1, 1, kernel_size=(4,4), stride=2, padding=1)

	def forward(self, x):
		x = self.upsampler(x)
		x = self.encoder(x).logits
		x = self.upscaler1(x)
		return x


model = CustomModel(scale_factor=2)
model.eval()
dummy_input = torch.randn(1, 1, 256, 256)
y_out = model(dummy_input)
y_out.shape

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b1 and are newly initialized: ['decode_head.linear_fuse.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.1.proj.bias', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.0.proj.weight', 'decode_head.classifier.weight', 'decode_head.batch_norm.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.batch_norm.weight', 'decode_head.linear_c.1.proj.weight', 'decode_head.batch_norm.running_var', 'decode_head.classifier.bias', 'decode_head.linear_c.2.proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b1 and are newly initialized because the shapes did not m

torch.Size([1, 1, 256, 256])